In [1]:
import pandas as pd
import tensorflow as tf

In [3]:
train_d = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\train.csv',header =None ,low_memory=False)
test_d = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\test.csv',header = None , low_memory =False)
sub_file = pd.read_csv(r'C:\Users\YATISH\Desktop\pucho\sample_submission.csv')

In [4]:
dff = train_d.copy()
dff.drop(0,inplace =True)
dff.reset_index(drop =True,inplace = True)

In [5]:
dft = test_d.copy()
dft.drop(0,inplace =True)
dft.reset_index(drop =True,inplace = True)

In [6]:
Y = dff.iloc[0:42000,0:1]
X = dff.iloc[0:42000,1:785]
X_test = dft.iloc[0:28000,0:784]

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
Y = ohe.fit_transform(Y).toarray()

C:\Users\YATISH\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\YATISH\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [8]:
n_input = 784
n_hidden1 =256
n_hidden2 =256
n_out =10


weights = {
    'h1': tf.Variable(tf.random_normal([n_input,n_hidden1])),
    'h2': tf.Variable(tf.random_normal([n_hidden1,n_hidden2])),
    'o': tf.Variable(tf.random_normal([n_hidden2,n_out]))
}

biases = {
    'h1' : tf.Variable(tf.random_normal([n_hidden1])),
    'h2' : tf.Variable(tf.random_normal([n_hidden2])),
    'o'  : tf.Variable(tf.random_normal([n_out]))
}

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
def forward_propagation(x,weights,biases):
    in_layer1 = tf.add(tf.matmul(x,weights['h1']),biases['h1'])
    out_layer1 = tf.nn.relu(in_layer1)
    
    in_layer2 = tf.add(tf.matmul(out_layer1,weights['h2']),biases['h2'])
    out_layer2 = tf.nn.relu(in_layer2)
    
    output = tf.add(tf.matmul(out_layer2,weights['o']),biases['o'])
    return output

In [10]:
x = tf.placeholder("float",[None,n_input])
y = tf.placeholder(tf.int32,[None,n_out])
pred = forward_propagation(x,weights,biases)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred,labels = y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
# batch gradient
batch_size = 100
for i in range(25):
    n_batches = len(X)//batch_size
    total_cost = 0
    b = 1
    for j in range(n_batches):
        b = batch_size*j
        batch_X = X[b:b+batch_size]
        batch_Y = Y[b:b+batch_size]
        c, _ = sess.run([cost,optimize],feed_dict ={x:batch_X,y:batch_Y})
        total_cost += c
    print(c)

1629.8673
828.27124
0.0
567.42535
669.22107
0.0
0.0
384.54352
516.5227
467.57062
0.0
168.95406
106.83828
265.04562
0.0
0.0
448.4174
0.0
0.0
19.215351
0.0
179.25383
148.70375
112.09547
24.099453


In [15]:
predictions = tf.argmax(pred,1)

pred_eval = sess.run(predictions , feed_dict = {x:X_test})
sess.close()


In [16]:
for i in range(len(sub_file)):
    sub_file.Label[i] = pred_eval[i]
sub_file.to_csv(r'C:\Users\YATISH\Desktop\pucho\submission.csv',index = False)

In [43]:
sub_file

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3
